In [2]:
-- hack for reload main.lua
function reload(name)
    package.loaded[name] = nil
    return require(name)
end

In [3]:
reload('main')

true	


In [4]:
-- load training data
dataset_inputs = torch.Tensor(read('X_train.csv',',',True))
dataset_outputs = torch.Tensor(read('y_train.csv',',',True)):t()[1]
dataset_filters = torch.Tensor(read('filters_train.csv',',',True))
tf_matrix = torch.Tensor(read('tf_matrix.csv',',',True)):t()

In [ ]:
-- make weighted tf model
h1 = - nn.CMul(1,1276)
h2 = - nn.Identity()
h3 = {h1,h2} - nn.MM()
--h4 = - nn.Identity()
--h5 = {h3,h4} - nn.MM()
h4 = h3 - nn.LogSoftMax()
model = nn.gModule({h1,h2}, {h4})

criterion = nn.ClassNLLCriterion()

In [ ]:
------------------------------------------------------------------------------------------------
--- Training Flow
------------------------------------------------------------------------------------------------

nepochs = 30
for i=1,nepochs do
    
    -- loop dataset
    tot_err = 0
    for j=1,9000 do
        
        -- load dataset line
        input = dataset_inputs[j]:view(1,1276)
        correct_class = dataset_outputs[j]
        
        -- forward pass
        pred = model:forward({input,tf_matrix})
        err = criterion:forward(pred, correct_class)
        tot_err = tot_err + err
        
        -- backward pass
        gradCriterion = criterion:backward(pred, correct_class)
        model:zeroGradParameters()
        model:backward(x, gradCriterion)
        learningRate = 1e-1
        model:updateParameters(learningRate)
        
    end

    print(tot_err)
end

In [6]:
--torch.save('good-model',model)

In [ ]:
---------------------------------------------------------------------------------------------
--- Test Flow
---------------------------------------------------------------------------------------------

-- model from training flow above
model = torch.load('good-model')

-- preprocessed features for test data are written to this file by Main.ipynb
-- one line per dataset entry
local file = assert(io.open('intermediate_info.txt', "r"))
local lines = file:lines()

-- loop dataset entries
for line in lines do
    parts = split(line,'|')
    correct_label = tonumber(parts[1])
    --print(correct_label)

    -- loop entity possibilities
    predictions = {}
    for i=2,#parts,5 do
        
        -- load dataset line
        ent_correct = parts[i]
        p_spelling = tonumber(parts[i+1])
        ent_score = tonumber(parts[i+2])
        data_input = torch.Tensor(split(parts[i+3],',',True)):view(1,1276)
        data_filter = torch.Tensor(split(parts[i+4],',',True))
        
        -- forward pass model then filter out labels that dont appear as relationship types for this ent
        pred = model:forward({data_input,tf_matrix})[1]
        pred_filtered = torch.cmul(pred,data_filter)
        --print(pred_filtered)
        -- loop relationship type classes
        for k=1,1531 do
            
            -- add an entry to predictions for this entity,relationship_type pair if an edge exists in freebase
            if pred_filtered[k]~=0 then 
                val = pred_filtered[k] + p_spelling * ent_score
                table.insert(predictions,{val,k,ent_correct})
                --print(k..' '..g[k]..' '..g_base[k]..' '..p) 
            end
        end
        
    end
    
    -- sort predictions by score
    function compare(a,b)
        return a[1] > b[1]
    end
    table.sort(predictions,compare)
    
    --for i=1,5 do
    --    print(predictions[i])
    --end
    
    -- see if top prediction has correct entity and relationship type
    if predictions[1][3]=='True' and predictions[1][2]==correct_label then
        print '$yes'
    else
        print '$no'
    end
    
end